In [ ]:
pip install evaluate

In [2]:
import spacy
import re
import pandas as pd
import numpy as np

import transformers
from sklearn.model_selection import train_test_split

import evaluate
from transformers import Trainer, TrainingArguments

c:\Users\itssh\anaconda\envs\Env_python3_10\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
#Define Pronoun Mapping
GENDERED_PRONOUNS_LIST = {
    "male": [
        ("he", "they"),
        ("him", "them"),
        ("his", "their"),
        ("himself", "themselves"),
        ("mr", "they"),
        ("men", "they")
    ],
    "female": [
        ("she", "they"),
        ("her", "them"),
        ("herself", "themselves"),
        ("women", "they"),
        ("ms", "they")
    ]
}

In [5]:
# Combine the two lists into a single mapping (if you want to mask both for neutrality)
GENDERED_PRONOUNS = dict(GENDERED_PRONOUNS_LIST['male'] + GENDERED_PRONOUNS_LIST['female'])

In [6]:
# Load SpaCy model
nlp = spacy.load("en_core_web_sm")

In [7]:
# Define a masking function
def mask_gendered_pronouns(text):
    doc = nlp(text)              # text assumed to be already lowercase
    masked_text = []

    for token in doc:
        token_lower = token.text
        if token_lower in GENDERED_PRONOUNS:
            masked_text.append(GENDERED_PRONOUNS[token_lower])
        else:
            masked_text.append(token_lower)
    
    return " ".join(masked_text)

In [8]:
# Load Bios dataset

df = pd.read_csv('df_accountant_lower.csv')
df.head()


,text,labels
0,he has been with the firm for seventeen years....,1
1,a conviction that organisations could do consi...,1
2,l. john walpole has been issued a tennessee li...,0
3,"most recently, he was deputy group cfo at inte...",1
4,he was employed with the very well-known compa...,1


In [9]:
# Apply the masking function
df["masked_text"] = df["text"].apply(mask_gendered_pronouns)


In [10]:
# Preview results
df[['text', 'masked_text', 'labels']].head()

,text,masked_text,labels
0,he has been with the firm for seventeen years....,they has been with the firm for seventeen year...,1
1,a conviction that organisations could do consi...,a conviction that organisations could do consi...,1
2,l. john walpole has been issued a tennessee li...,l. john walpole has been issued a tennessee li...,0
3,"most recently, he was deputy group cfo at inte...","most recently , they was deputy group cfo at i...",1
4,he was employed with the very well-known compa...,they was employed with the very well - known c...,1


In [11]:
# Download bert-base-uncased model for binary classification
model = transformers.AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
# Prepare training and test data with 80-20 split


train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [13]:
# Prepare text data for training using bert tokenizer
from transformers import AutoTokenizer
import datasets

train_dataset = datasets.Dataset.from_pandas(train_df)
test_dataset = datasets.Dataset.from_pandas(test_df)

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def encode(examples):
    return tokenizer(examples['text'], truncation = True, padding='max_length')

train_dataset = train_dataset.map(encode, batched=True)
test_dataset = test_dataset.map(encode, batched=True)


dataset = datasets.DatasetDict({"train": train_dataset, "test": test_dataset})

Map: 100%|██████████| 200/200 [00:00<00:00, 2389.40 examples/s]


In [14]:


metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    # convert the logits to their predicted class
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
pip install transformers[torch]

In [16]:
import os
os.environ['WANDB_DISABLED'] = 'true'

In [ ]:
# Fine-tune model for classification


training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size= 8
)

# train model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
    compute_metrics=compute_metrics
)

trainer.train()

model.eval()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
 61%|██████▏   | 184/300 [40:20<27:09, 14.05s/it] 

In [ ]:
# Empty GPU cache to free up space
import torch
import gc
torch.cuda.empty_cache()
torch.cuda.memory_allocated()
gc.collect()

In [ ]:
# Compute accuracy, recall, precision and F1 of trained model for sci-kit learn
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

# Compute for training data
y_pred = trainer.predict(dataset['train'])
y_pred = np.argmax(y_pred.predictions, axis=-1)
y_true = dataset['train']['labels']
print('Training data:')
print('Accuracy:', accuracy_score(y_true, y_pred))
print('Recall:', recall_score(y_true, y_pred))
print('Precision:', precision_score(y_true, y_pred))
print('F1:', f1_score(y_true, y_pred))

# Compute for test data
y_pred = trainer.predict(dataset['test'])
y_pred = np.argmax(y_pred.predictions, axis=-1)
y_true = dataset['test']['labels']
print('Test data:')
print('Accuracy:', accuracy_score(y_true, y_pred))
print('Recall:', recall_score(y_true, y_pred))
print('Precision:', precision_score(y_true, y_pred))
print('F1:', f1_score(y_true, y_pred))


In [ ]:
# View confusion matrix for test data
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

y_pred = trainer.predict(dataset['test'])
y_pred = np.argmax(y_pred.predictions, axis=-1)
y_true = dataset['test']['labels']
cm = confusion_matrix(y_true, y_pred)
sns.heatmap(cm, annot=True, fmt='d')
# label plot
plt.xlabel('Predicted')
plt.ylabel('True')

plt.show()

In [ ]:
import torch
from torch import nn

predictions = torch.from_numpy(trainer.predict(dataset['test']).predictions)
pred_probs = nn.functional.softmax(predictions, dim=-1).detach().numpy()

In [ ]:
### Sort test samples according to male probability
probs = pred_probs[:, 1]
sorted_indices = np.argsort(probs)

# Reverse the sortation
sorted_indices = sorted_indices[::-1]

test_sorted = dataset['test'].select(sorted_indices)

df_sorted_male = pd.DataFrame({'text': test_sorted['text'], 'labels': test_sorted['labels'], 'prob': probs[sorted_indices]})
df_sorted_male.head()

LIME